In [266]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [267]:
training_dataset = pd.read_csv("train.csv")

In [268]:
training_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [269]:
training_dataset = pd.concat([training_dataset,
                              pd.get_dummies(training_dataset["Sex"]),
                              pd.get_dummies(training_dataset["Embarked"]),
                             ], axis=1)

In [270]:
training_dataset.drop(columns=["PassengerId", "Name", "Ticket", "Sex", "Cabin", "Embarked"], inplace=True)

In [271]:
training_dataset["Age"] = training_dataset["Age"].fillna(training_dataset["Age"].mean())

In [272]:
Y = training_dataset.loc[:, "Survived"]
X = training_dataset.iloc[:, 1:]

In [273]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [274]:
rf_clsf = RandomForestClassifier(n_estimators=100)

In [275]:
rf_clsf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [276]:
y_pred = rf_clsf.predict(X_test)

In [277]:
metrics.accuracy_score(y_test, y_pred)

0.7982062780269058

In [278]:
feature_imp = pd.Series(rf_clsf.feature_importances_, index=training_dataset.columns[1:]).sort_values(ascending=False)

In [279]:
feature_imp

Fare      0.255690
Age       0.249104
female    0.160115
male      0.116407
Pclass    0.090604
SibSp     0.048566
Parch     0.040051
C         0.015865
S         0.015765
Q         0.007834
dtype: float64

In [319]:
test_dataset = pd.read_csv("test.csv")

In [320]:
test_dataset = pd.concat([test_dataset,
                              pd.get_dummies(test_dataset["Sex"]),
                              pd.get_dummies(test_dataset["Embarked"]),
                             ], axis=1)

In [321]:
test_df = test_dataset.drop(columns=["PassengerId", "Name", "Ticket", "Sex", "Cabin", "Embarked"])

In [322]:
test_df["Age"] = test_df["Age"].fillna(test_df["Age"].mean())

In [323]:
test_df["Fare"] = test_df["Fare"].fillna(test_df["Fare"].mean())

In [324]:
training_dataset.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [325]:
test_df.head()

,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,3,34.5,0,0,7.8292,0,1,0,1,0
1,3,47.0,1,0,7.0000,1,0,0,0,1
2,2,62.0,0,0,9.6875,0,1,0,1,0
3,3,27.0,0,0,8.6625,0,1,0,0,1
4,3,22.0,1,1,12.2875,1,0,0,0,1


In [326]:
len(test_df.columns), len(training_dataset.columns)

(10, 11)

In [327]:
X_dev = test_df.iloc[:, :]

In [328]:
test_df["Survived"] = rf_clsf.predict(X_dev)

In [329]:
test_df["PassengerId"] = test_dataset["PassengerId"]

In [330]:
test_df[["PassengerId", "Survived"]].to_csv("submission.csv", index=False)